In [ ]:
import os
os.chdir("..")

In [ ]:
import h5py
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sb
from scipy.stats import beta as BetaDistr

In [ ]:
file = h5py.File("data/fitness.h5")

path = {}
for name,group in file.items():
	shapes = [run["infections"].shape[0] for run in group.values()]
	seedtolook = list(group.keys())[np.argmax(shapes)]
	path[name] = f"{name}/{seedtolook}"

allfitness = {name : file[f"{simulation}/fitness"][:] for name,simulation in path.items()}
infections = {name : file[f"{simulation}/infections"].fields(["t","mut"])[:] for name,simulation in path.items()}
mutation_rates = {name : file[simulation].attrs["mutation_rate"] for name,simulation in path.items()}
fitness_params = {name : {key:val for key,val in file[simulation].attrs.items() if "fitness_" in key} for name,simulation in path.items()}

file.close()

In [ ]:
fitness_params

In [ ]:
evolvingfitness = {}
for name,allfit in allfitness.items():
	allfit = pd.DataFrame.from_records(allfit, index="id")
	df = pd.DataFrame.from_records(infections[name], index="mut")
	df["phi"] = allfit.loc[df.index]
	df.reset_index(inplace=True)
	evolvingfitness[name] = df

In [ ]:
print(f"Mutation rate = {mutation_rates[name]}\nFitness std = {fitness_params[name]['fitness_scale']}")

In [ ]:
nrows = len(evolvingfitness)
fig = plt.figure(figsize=(8,4*nrows))
grid = fig.add_gridspec(nrows, 2, width_ratios=(3,1), wspace=0.05, hspace=0.2)
for name,df in evolvingfitness.items():
	i = int(name)
	axbig = fig.add_subplot(grid[i,0])
	sb.histplot(data=df, x="t", y="phi", bins=50, ax=axbig)
	#axbig.set_title(f"Mutation rate={mutation_rates[name]}  fitness std={fitness_params[name]['fitness_std']}")
	xdisplay, ydisplay = axbig.transData.transform((0.1, 0.9))
	axbig.text(0.05, 0.8, f"Fitness beta = {fitness_params[name]['fitness_beta']}\nFitness scale = {fitness_params[name]['fitness_scale']}", transform=axbig.transAxes)
	axbig.set_ylabel("Fitness")
	axbig.set_xlabel("t")
	axbig.set_xlim(left=0)
	axmarg = fig.add_subplot(grid[i,1], sharey=axbig)
	sb.histplot(y=allfitness[name]["phi"], bins=50, stat="density", alpha=0.5, ax=axmarg)
	alpha = fitness_params[name]["fitness_alpha"]
	beta = fitness_params[name]["fitness_beta"]
	scale = fitness_params[name]["fitness_scale"]
	loc = fitness_params[name]["fitness_mean"] - alpha / (alpha + beta) * scale
	xs = np.linspace(allfitness[name]["phi"].min(), allfitness[name]["phi"].max(), 1001)
	axmarg.plot(BetaDistr.pdf(xs, alpha, beta, loc=loc, scale=scale), xs)
	axmarg.tick_params(axis="y", labelleft=False)
	axmarg.set_xlabel("")
	axmarg.set_xticks([])
fig.subplots_adjust()
fig.savefig("analysis/fitness_evolution.png")

In [ ]:
mutation_rates

In [ ]:
from scipy.stats import gamma, beta, norm
from scipy.optimize import root_scalar
from scipy.integrate import quad

In [ ]:
quad(f, 1, 2)[0]

In [ ]:
fig, axs = plt.subplots(3, 1, figsize=(6,6), sharex=True)

xs = np.linspace(-1.05,1.05,10001)
xsplus = xs[xs>0]
mean = 0
std = 0.2

k = 2
theta = std / np.sqrt(k)
def f(x):
	return gamma.pdf(x, k, scale=theta, loc=mean-k*theta)
axs[0].plot(xs, f(xs))
P = quad(f, 0, 2)[0]
axs[0].fill_between(xsplus, gamma.pdf(xsplus, k, scale=theta, loc=mean-k*theta), alpha=0.3, label=r"$P(x > 1)=$"+f"{P:.2f}")
axs[0].legend(title="Gamma")

def f(x):
	return norm.pdf(x, scale=std, loc=mean)
axs[1].plot(xs, f(xs))
P = quad(f, 0, 2)[0]
axs[1].fill_between(xsplus, norm.pdf(xsplus, scale=std, loc=mean), alpha=0.3, label=r"$P(x > 1)=$"+f"{P:.2f}")
axs[1].legend(title="Normal")

a = 2
b = 50
scale = std
def f(x):
	return beta.pdf(x, a, b, loc=mean-a/(a+b)*scale, scale=scale)
axs[2].plot(xs, f(xs))
P = quad(f, 0, 2)[0]
axs[2].fill_between(xsplus, beta.pdf(xsplus, a, b, loc=mean-a/(a+b)*scale, scale=scale), alpha=0.3, label=r"$P(x > 1)=$"+f"{P:.2f}")
axs[2].legend(title="Beta")

for ax in axs:
	ax.grid()
	ax.set_yticks([])

fig.subplots_adjust(hspace=0)
#fig.tight_layout()

In [ ]:
scale = 2

In [ ]:
somevalues = beta.rvs(a, b, loc=mean-a/(a+b)*scale, scale=scale, size=int(1e6))
plt.hist(somevalues, bins=500, density=True)
plt.axvline(somevalues.mean(), color="k", linestyle="--")
plt.plot(xs, beta.pdf(xs, a, b, loc=mean-a/(a+b)*scale, scale=scale))
plt.xlim([-0.01,0.04])

In [ ]:
from TravelAndMutate.distributions import BetaDistribution
from TravelAndMutate.randominterface import NumpyRandomGenerator

In [ ]:
eng = NumpyRandomGenerator(156432)

In [ ]:
scales = [1,2,4]
for scale in scales:
	plt.plot(xs, beta.pdf(xs, a, b, loc=mean-a/(a+b)*scale, scale=scale))
plt.xlim([-0.1,0.4])

In [ ]:
scale = 1
bs = [3,6,10,50]
for b in bs:
	def f(x):
		return beta.pdf(x, a, b, loc=mean-a/(a+b)*scale, scale=scale)
	P = beta.median(a, b, loc=mean-a/(a+b)*scale, scale=scale)
	p = beta.ppf(0.95, a, b, loc=mean-a/(a+b)*scale, scale=scale)
	plt.plot(xs, f(xs), label=f"{P:.2f} ; {(a-1)/(a+b-2)-a/(a+b):.2f} ; {p:.3f}")
	plt.axvline((a-1)/(a+b-2)-a/(a+b), linestyle=":")
plt.legend()
plt.xlim([-0.5,0.4])
plt.ylim([0,5])

In [ ]:
alphas = np.geomspace(0.00001,100,100)
betas = alphas
stds = np.empty(len(alphas)*len(betas))
i = 0
for alpha in alphas:
	for beta in betas:
		stds[i] = np.sqrt(alpha * beta / (alpha+beta)**2 / (alpha+beta+1))
		i = i + 1

In [ ]:
values = pd.DataFrame({"std":stds}, index=pd.MultiIndex.from_product([alphas,betas], names=["alpha","beta"])).reset_index()

In [ ]:
sb.heatmap(data=values.pivot(index="alpha", columns="beta", values="std"))

In [ ]:
values["std"].argmax()

In [ ]:
values.iloc[0]